In [1]:
'''
Kappa coefficient is a statistic which measures inter-rater agreement for qualitative (categorical) items.
Images have five possible ratings, 0,1,2,3,4.  
Each image is characterized by a tuple (ea,eb), which corresponds to its scores by Rater A (human) and Rater B (predicted).
'''
#EXAMPLE TO TEST THE KAPPA GIVEN TWO npARRAYS

# O = Observed Matriz O(i,j) -> Amount of images that received a rating i by A and a rating j by B

import numpy as np
import math
np.random.seed(1337) 

# ~50% correlated ratings
N = int(1e6)
A = np.random.choice(5,size=(N))
B = A*1
ind = np.random.rand(N) >= 0.5
B[ind] = np.random.choice(5,size=(np.sum(ind)))

 #input Example array
# y_true = np.array([1,1,4,4,1,0,2])
# y_pred = np.array([1,2,4,3,1,1,2])

y_true = A
y_pred = B
print(y_true)
print(y_pred)



posRatings = 5
O  = np.zeros((posRatings,posRatings) ,dtype="int")
W = E = np.zeros((posRatings,posRatings) ,dtype="float32")
A = np.zeros((posRatings,1) ,dtype="float32")
B = np.zeros((posRatings,1) ,dtype="float32")

for i in xrange(posRatings):
    A[i] = np.sum((y_true[:]==i))
    B[i] = np.sum((y_pred[:]==i))
    for j in xrange(posRatings):
        O[i,j] = np.sum((y_true[:]==i) & (y_pred[:]==j))
        W[i,j] = math.pow((i-j), 2)/math.pow((posRatings-1), 2)

E = np.outer(B,A)
#Normalization
E = (E*np.sum(O))/np.sum(E)

k = 1 -(np.sum(W*O))/(np.sum(W*E))

print "O:","\n" ,O
print "W:","\n" ,W
print "E:","\n" , E
print "k:","\n",k


[4 0 1 ..., 1 0 1]
[4 0 3 ..., 2 3 1]
O: 
[[119593  20142  20049  20434  19970]
 [ 19809 120233  19990  20281  20086]
 [ 19964  19997 119806  19933  20116]
 [ 20173  19950  19997 119856  19910]
 [ 19795  19892  19953  19963 120108]]
W: 
[[ 0.      0.0625  0.25    0.5625  1.    ]
 [ 0.0625  0.      0.0625  0.25    0.5625]
 [ 0.25    0.0625  0.      0.0625  0.25  ]
 [ 0.5625  0.25    0.0625  0.      0.0625]
 [ 1.      0.5625  0.25    0.0625  0.    ]]
E: 
[[ 39904.27402828  39946.3317537   39830.12004884  39844.07511998
   39809.18949012]
 [ 40080.43888146  40122.68092686  40005.95722203  40019.97373317
   39984.93655133]
 [ 39996.56099861  40038.71702802  39922.23498718  39936.22282632
   39901.25527647]
 [ 40131.08591835  40173.38530775  40056.51005093  40070.54294607
   40035.46070822]
 [ 40075.63427375  40117.87222316  40001.16080633  40015.17731747
   39980.14423162]]
k: 
0.499208179069


In [2]:
#METHOD WITH NP ARRAYS, FINDING THE LOSS (Doesn't work in the network because it works with tensonrs instead of npArrays)
# from keras import backend as K
import numpy as np
import math

def kappa(y_true, y_pred): #Receive two np.array()
    
    #Flatten to became the inputs in singles lines of data
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    
    #Var init
    posRatings = 5
    O  = np.zeros((posRatings,posRatings) ,dtype="int")
    W = E = np.zeros((posRatings,posRatings) ,dtype="float32")
    A = np.zeros((posRatings,1) ,dtype="float32")
    B = np.zeros((posRatings,1) ,dtype="float32")
    
    #Kappa calculation
    for i in xrange(posRatings):
        #print(y_true[:]==i)
        A[i] = np.sum((y_true[:]==i))
        #print(A)
        B[i] = np.sum((y_pred[:]==i))
        
        for j in xrange(posRatings):
            O[i,j] = np.sum((y_true[:]==i) & (y_pred[:]==j))
            W[i,j] = math.pow((i-j), 2)/math.pow((posRatings-1), 2)

    E = np.outer(B,A)
    #Normalization
    E = (E*np.sum(O))/np.sum(E)

    k = 1 -(np.sum(W*O))/(np.sum(W*E))

    return k

def kappa_loss(y_true, y_pred):
    return (1 - kappa(y_true, y_pred))

    
x = np.array([(1,2) , (5,2), (4,9)])
f = np.array([(1,2) , (5,3), (4,9)])
print(x)
kappa_loss(x,f)

[[1 2]
 [5 2]
 [4 9]]


0.099999999999999978

In [3]:
#KAPPA WITH TENSORS (Doesnt work inside teh netwok, just for test with new tensors because 
#it requires create a graph when the method eval() is used but in the network is not possible create new graph inside the network graph)
def kappa(y_true, y_pred): #Receive two np.array()

    #Flatten to became the inputs in singles lines of data
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)

    print(y_true.type) 
    #Var init
    posRatings = 5
    O  = np.zeros((posRatings,posRatings) ,dtype="int")
    W = E = np.zeros((posRatings,posRatings) ,dtype="float32")
    A = np.zeros((posRatings,1) ,dtype="float32")
    B = np.zeros((posRatings,1) ,dtype="float32")
    #Kappa calculation
    for i in xrange(posRatings):
        A[i] = K.sum((y_true.eval())[:]==i).eval()
        B[i] = K.sum((y_pred.eval())[:]==i).eval()
        for j in xrange(posRatings):
            O[i,j] = np.sum(((y_true.eval())[:]==i ) & ((y_pred.eval())[:]==j))
            W[i,j] = math.pow((i-j), 2)/math.pow((posRatings-1), 2)

    E = np.outer(B,A)
    #Normalization
    E = (E*np.sum(O))/np.sum(E)

    k = 1 -(np.sum(W*O))/(np.sum(W*E))

    k = K.variable(value=k)
    return k

def kappa_loss(y_true, y_pred):
    print("y_true", y_true.type)
    print("y_pred", y_pred)
    
    
    return (1.-kappa(y_true, y_pred))

In [4]:
#EXAMPLE OF A LOSS FUNCTION WORKING WITH TENSORS
import theano
import theano.tensor as T

epsilon = 1.0e-9
def custom_objective(y_true, y_pred):
    '''Just another crossentropy'''
    y_pred = T.clip(y_pred, epsilon, 1.0 - epsilon)
    y_pred /= y_pred.sum(axis=-1, keepdims=True)
    cce = T.nnet.categorical_crossentropy(y_pred, y_true)
    print("cce", cce.type)
    return T.sum(y_pred)


Using gpu device 0: GeForce GTX 980 Ti (CNMeM is disabled, cuDNN 4007)


In [5]:
#TRY OF KAPPA WITH TENSORS, without eval... it doenst work because it requires create a graph/sesion 
#to test and visualizate the calculation... require too much knowlegbe about backend (graph, tensors, sesions)  
#that its not realle relevant in this moment
from keras import backend as K
import numpy as np
import math


y_true = np.array([(1,2) , (5,2), (4,9)])
y_pred = np.array([(1,2) , (5,3), (4,9)])

y_true = K.variable(value=y_true)
y_pred = K.variable(value=y_pred)
print(y_true)


#Flatten to became the inputs in singles lines of data
y_true = K.flatten(y_true)
y_pred = K.flatten(y_pred)

print(y_true.type) 
#Var init
posRatings = 5
O  = np.zeros((posRatings,posRatings) ,dtype="int")
W = E = np.zeros((posRatings,posRatings) ,dtype="float32")
A = np.zeros((posRatings,1) ,dtype="float32")
B = np.zeros((posRatings,1) ,dtype="float32")
#Kappa calculation
for i in xrange(posRatings):
    A[i] = K.sum((y_true==i)*1).eval()
    B[i] = K.sum((y_pred==i)*1).eval()
    for j in xrange(posRatings):
        O[i,j] = np.sum((y_true==i ) & ((y_pred==j)))
        W[i,j] = math.pow((i-j), 2)/math.pow((posRatings-1), 2)

E = np.outer(B,A)
#Normalization
E = (E*np.sum(O))/np.sum(E)

k = 1 -(np.sum(W*O))/(np.sum(W*E))
print(k)
k = K.variable(value=k)
print(K.get_value(k))

Using Theano backend.


<CudaNdarrayType(float32, matrix)>
TensorType(float32, vector)
nan
nan


In [6]:
#TEST WITH TENSORS COMPARED WITH npARRAYS

from keras import backend as K
import numpy as np

var = K.zeros(shape=(3, 4))
var[0,0]

val = np.random.random((3, 4, 5))
var = K.variable(value=val)

print("value:" ,K.get_value(var))

#-------------------------------------------------------------------

y_true = np.array([(1,2) , (5,2), (4,9)])
y_pred = np.array([(1,2) , (5,3), (4,9)])

y_true = K.variable(value=y_true)
y_pred = K.variable(value=y_pred)
print(y_true)


#Flatten to became the inputs in singles lines of data
y_true = K.flatten(y_true)
y_pred = K.flatten(y_pred)
print(y_true.eval())

print  (((y_true.eval())==1))
print  (y_true==i)
print(K.sum((y_true.eval())[:]==2).eval())


np.sum(((y_true.eval())[:]==i ) & ((y_pred.eval())[:]==j))

('value:', array([[[ 0.18105799,  0.61045426,  0.78873605,  0.80182117,  0.76779622],
        [ 0.45899862,  0.39655864,  0.52267075,  0.89335215,  0.31162602],
        [ 0.71643686,  0.13007   ,  0.24909572,  0.42383876,  0.60103661],
        [ 0.72988379,  0.93980372,  0.83842373,  0.13508058,  0.21601191]],

       [[ 0.52879375,  0.82019538,  0.04725382,  0.24159412,  0.24546787],
        [ 0.91953689,  0.58379048,  0.4843815 ,  0.39862552,  0.73301393],
        [ 0.63727134,  0.85996878,  0.0436643 ,  0.7668137 ,  0.01840357],
        [ 0.13605583,  0.02648348,  0.72060406,  0.69427705,  0.33143523]],

       [[ 0.24267319,  0.00968867,  0.58235574,  0.07816658,  0.98280871],
        [ 0.13356765,  0.43699944,  0.67111415,  0.77977091,  0.55458999],
        [ 0.62704247,  0.58929062,  0.28022635,  0.90689701,  0.78429508],
        [ 0.42736939,  0.20563412,  0.51787698,  0.20004272,  0.37634695]]], dtype=float32))
<CudaNdarrayType(float32, matrix)>
[ 1.  2.  5.  2.  4.  9.]
[ True

1